# Keras Model

In [5]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("Data/sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2
)

# Load the separate test data set
test_data_df = pd.read_csv("Data/sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=2)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

Epoch 1/50
 - 0s - loss: 0.0198
Epoch 2/50
 - 0s - loss: 0.0019
Epoch 3/50
 - 0s - loss: 6.9092e-04
Epoch 4/50
 - 0s - loss: 4.0879e-04
Epoch 5/50
 - 0s - loss: 2.4531e-04
Epoch 6/50
 - 0s - loss: 1.8760e-04
Epoch 7/50
 - 0s - loss: 1.2512e-04
Epoch 8/50
 - 0s - loss: 9.5138e-05
Epoch 9/50
 - 0s - loss: 8.1931e-05
Epoch 10/50
 - 0s - loss: 7.8103e-05
Epoch 11/50
 - 0s - loss: 6.7223e-05
Epoch 12/50
 - 0s - loss: 4.9072e-05
Epoch 13/50
 - 0s - loss: 5.0025e-05
Epoch 14/50
 - 0s - loss: 6.2394e-05
Epoch 15/50
 - 0s - loss: 4.7856e-05
Epoch 16/50
 - 0s - loss: 4.5250e-05
Epoch 17/50
 - 0s - loss: 3.9453e-05
Epoch 18/50
 - 0s - loss: 3.8667e-05
Epoch 19/50
 - 0s - loss: 3.9599e-05
Epoch 20/50
 - 0s - loss: 3.3455e-05
Epoch 21/50
 - 0s - loss: 3.2035e-05
Epoch 22/50
 - 0s - loss: 3.7781e-05
Epoch 23/50
 - 0s - loss: 3.9308e-05
Epoch 24/50
 - 0s - loss: 3.2197e-05
Epoch 25/50
 - 0s - loss: 2.5709e-05
Epoch 26/50
 - 0s - loss: 2.4020e-05
Epoch 27/50
 - 0s - loss: 3.7908e-05
Epoch 28/50
 - 0s 

In [ ]:
model_builder = tf.saved_model.builder.SavedModelBuilder("exported_keras_model")

inputs = {
    'input': tf.saved_model.utils.build_tensor_info(model.input)
}
outputs = {
    'earnings': tf.saved_model.utils.build_tensor_info(model.output)
}

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()
